In [1]:
import pandas as pd
import numpy as np

In [25]:
data_path="../data/"
outputs_path="../outputs/"
test_set=pd.read_csv(outputs_path+"general/test_set_final.csv")
train_set=pd.read_csv(outputs_path+"general/train_set_final.csv")
dev_set=pd.read_csv(outputs_path+"general/dev_set_final.csv")

df_g=pd.read_csv("../sg_match/progress_singular_gpt3.csv")
low_ocurrence=pd.read_csv(data_path+"test_data/low_occurrence.csv")


In [26]:
low_ocurrence=low_ocurrence.rename(columns={'Most Related SG goal':'goal1_raw',
       '(if exists) 2nd Related SG Goal':'goal2_raw', '(if exists) 3rd Related SG Goal':'goal3_raw'})
low_ocurrence=low_ocurrence.rename(columns={"Zhijing's annotation of SG_or_not":"label"})
low_ocurrence["label"]=low_ocurrence["label"].fillna(0)
low_ocurrence.abstract_clean=low_ocurrence.abstract_clean.fillna('')
low_ocurrence=low_ocurrence.assign(text=low_ocurrence.title_clean+". "+low_ocurrence.abstract_clean)
low_ocurrence=low_ocurrence.iloc[:97]
df_all_goals=pd.concat([dev_set,train_set,test_set,low_ocurrence])

In [27]:
def extract_predictions_gpt3(df):
    df=df.assign(sdg1=np.where(df['GPT3_response'].str.lower().str.contains("goal 1 |goal 1:|poverty"),1,0))
    df=df.assign(sdg2=np.where(df['GPT3_response'].str.lower().str.contains("goal 2|hunger"),1,0))
    df=df.assign(sdg3=np.where(df['GPT3_response'].str.lower().str.contains("goal 3|health"),1,0))
    df=df.assign(sdg4=np.where(df['GPT3_response'].str.lower().str.contains("goal 4|education"),1,0))
    df=df.assign(sdg5=np.where(df['GPT3_response'].str.lower().str.contains("goal 5|gender"),1,0))
    df=df.assign(sdg6=np.where(df['GPT3_response'].str.lower().str.contains("goal 6|clean water"),1,0))
    df=df.assign(sdg7=np.where(df['GPT3_response'].str.lower().str.contains("goal 7|clean energy"),1,0))
    df=df.assign(sdg8=np.where(df['GPT3_response'].str.lower().str.contains("goal 8|decent work"),1,0))
    df=df.assign(sdg9=np.where(df['GPT3_response'].str.lower().str.contains("goal 9|industry|innovation"),1,0))
    df=df.assign(sdg10=np.where(df['GPT3_response'].str.lower().str.contains("goal 10|inequal"),1,0))
    df=df.assign(sdg11=np.where(df['GPT3_response'].str.lower().str.contains("goal 11|sustainable cities"),1,0))
    df=df.assign(sdg12=np.where(df['GPT3_response'].str.lower().str.contains("goal 12|responsible consumption"),1,0))
    df=df.assign(sdg13=np.where(df['GPT3_response'].str.lower().str.contains("goal 13|climate"),1,0))
    df=df.assign(sdg14=np.where(df['GPT3_response'].str.lower().str.contains("goal 14|life below water"),1,0))
    df=df.assign(sdg15=np.where(df['GPT3_response'].str.lower().str.contains("goal 15|life on land"),1,0))
    df=df.assign(sdg16=np.where(df['GPT3_response'].str.lower().str.contains("goal 16|peace|justice"),1,0))
    df=df.assign(sdg17=np.where(df['GPT3_response'].str.lower().str.contains("goal 17|partnership"),1,0))
    return df

In [28]:
df_g=extract_predictions_gpt3(df_g)

In [29]:
mapping_sdg={'sdg1': 'No Poverty',
 'sdg2': 'Zero Hunger',
 'sdg3': 'Good Health and Well-Being',
 'sdg4': 'Quality Education',
 'sdg5': 'Gender Equality',
 'sdg6': 'Clean Water and Sanitation',
 'sdg7': 'Affordable and Clean Energy',
 'sdg8': 'Decent Work and Economic Growth',
 'sdg9': 'Industry, Innovation and Infrastructure',
 'sdg10': 'Reduced Inequalities',
 'sdg11': 'Sustainable Cities and Communities',
 'sdg12': 'Responsible Consumption and Production',
 'sdg13': 'Climate Action',
 'sdg14': 'Life Below Water',
 'sdg15': 'Life on Land',
 'sdg16': 'Peace, Justice and Strong Institutions',
 'sdg17': 'Partnership for the Goals'
}

In [34]:
df_g=pd.melt(df_g,id_vars=['ID','year'],value_vars=['sdg1', 'sdg2', 'sdg3', 'sdg4', 'sdg5',
       'sdg6', 'sdg7', 'sdg8', 'sdg9', 'sdg10', 'sdg11', 'sdg12', 'sdg13',
       'sdg14', 'sdg15', 'sdg16', 'sdg17'],value_name='goal_binary',var_name='goal_number')

In [37]:
df_g=df_g.loc[df_g.goal_binary==1]

In [38]:
df_g['Goal']=df_g.goal_number.replace(mapping_sdg)

/cluster/project/sachan/fgonzalez/ie/lib64/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_all_goals.goal1_raw=df_all_goals.goal1_raw.fillna('')
df_all_goals.goal2_raw=df_all_goals.goal2_raw.fillna('')
df_all_goals.goal3_raw=df_all_goals.goal3_raw.fillna('')
df_all_goals=df_all_goals.assign(goal1=np.where(df_all_goals['goal1_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("marine_life"),'Life Below Water',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal1_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))

df_all_goals=df_all_goals.assign(goal2=np.where(df_all_goals['goal2_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("marine_life"),'Life Below Water',         
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal2_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))

df_all_goals=df_all_goals.assign(goal3=np.where(df_all_goals['goal3_raw'].str.lower().str.contains("education"),'Quality Education',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("poverty"),'No Poverty',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("hunger"),'Zero Hunger',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("clean_water"),'Clean Water and Sanitation',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("clean_energy"),'Affordable and Clean Energy',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("life_land"),'Life on Land',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("marine_life"),'Life Below Water',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("health"),'Good Health and Well-Being',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("climate"),'Climate Action',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("peace|privacy|disinformation_and_fake_news|deception|hate"),'Peace, Justice and Strong Institutions',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("social biases|race & identity"),'Reduced Inequalities',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("industry|innovation|research"),'Industry, Innovation and Infrastructure',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("sustainable cities|sustainable_cities"),'Sustainable Cities and Communities',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("gender"),'Gender Equality',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("decent work|decent_work_and_economy"),'Decent Work and Economic Growth',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("partnership"),'Partnership for the Goals',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("responsible_consumption_and_production"),'Responsible Consumption and Production',
                    np.where(df_all_goals['goal3_raw'].str.lower().str.contains("reduced|social_equality"),'Reduced Inequalities',''
                          )))))))))))))))))))


In [19]:
df_group=pd.concat([df_all_goals.loc[:,['ID','goal1','year']],df_all_goals.loc[:,['ID','goal2','year']].rename(columns={'goal2':'goal1'}),df_all_goals.loc[:,['ID','goal3','year']].rename(columns={'goal3':'goal1'})])

In [20]:
df_group=df_group.loc[df_group.goal1!='']

In [21]:
df_group=df_group.rename(columns={'goal1':'Goal'})

In [42]:
df_papers=pd.concat([df_group,df_g]).reset_index(drop=True)

In [ ]:
df_papers.year.apply(int)

In [45]:
df_agg=df_papers.groupby(['Goal','year']).ID.count().reset_index()


df_agg=df_agg.assign(total=df_agg.groupby(['year']).ID.transform(lambda x:x.sum()))

df_agg=df_agg.assign(total_max=df_agg.groupby(['year']).ID.transform(lambda x:x.max()))

df_agg=df_agg.assign(proportion=df_agg.ID/df_agg.total)
df_agg=df_agg.assign(proportion_max=df_agg.ID/df_agg.total_max)

df_agg=df_agg.rename(columns={'ID':'papers'})

In [49]:
df_agg

,Goal,year,papers,total,total_max,proportion,proportion_max
0,Affordable and Clean Energy,2019.0,2,813,284,0.002460,0.007042
1,Affordable and Clean Energy,2020.0,1,1445,443,0.000692,0.002257
2,Affordable and Clean Energy,2021.0,1,1207,441,0.000829,0.002268
3,Clean Water and Sanitation,2017.0,1,431,157,0.002320,0.006369
4,Clean Water and Sanitation,2020.0,1,1445,443,0.000692,0.002257
...,...,...,...,...,...,...,...
362,Zero Hunger,2009.0,1,148,69,0.006757,0.014493
363,Zero Hunger,2012.0,1,310,97,0.003226,0.010309
364,Zero Hunger,2018.0,1,585,189,0.001709,0.005291
365,Zero Hunger,2019.0,2,813,284,0.002460,0.007042


## all years

In [50]:
df_agg_total=df_papers.groupby(['Goal']).ID.count().reset_index()
df_agg_total=df_agg_total.assign(total=df_agg_total.ID.sum())
df_agg_total=df_agg_total.assign(proportion=df_agg_total.ID/df_agg_total.total)
df_agg_total=df_agg_total.assign(total_max=df_agg_total.ID.max())
df_agg_total=df_agg_total.assign(proportion_max=df_agg_total.ID/df_agg_total.total_max)
df_agg_total=df_agg_total.rename(columns={'ID':'papers'})

df_agg_total=df_agg_total.assign(year="All years")

In [51]:
df_agg_final=pd.concat([df_agg,df_agg_total])

In [52]:
df_agg.loc[df_agg.year==2010]

,Goal,year,papers,total,total_max,proportion,proportion_max
7,Climate Action,2010.0,1,226,81,0.004425,0.012346
48,Decent Work and Economic Growth,2010.0,13,226,81,0.057522,0.160494
67,Gender Equality,2010.0,1,226,81,0.004425,0.012346
110,Good Health and Well-Being,2010.0,81,226,81,0.358407,1.000000
154,"Industry, Innovation and Infrastructure",2010.0,6,226,81,0.026549,0.074074
175,Life on Land,2010.0,1,226,81,0.004425,0.012346
192,Partnership for the Goals,2010.0,1,226,81,0.004425,0.012346
222,"Peace, Justice and Strong Institutions",2010.0,29,226,81,0.128319,0.358025
273,Quality Education,2010.0,63,226,81,0.278761,0.777778
303,Reduced Inequalities,2010.0,5,226,81,0.022124,0.061728


In [7]:
df_agg.loc[df_agg.year==2010]

,Goal,year,papers,total,total_max,proportion,proportion_max
13,Affordable and Clean Energy,2010,5,236,90,0.021186,0.055556
55,Climate Action,2010,1,236,90,0.004237,0.011111
94,Decent Work and Economic Growth,2010,13,236,90,0.055085,0.144444
119,Gender Equality,2010,4,236,90,0.016949,0.044444
155,Good Health and Well-Being,2010,64,236,90,0.271186,0.711111
197,"Industry, Innovation and Infrastrucure",2010,10,236,90,0.042373,0.111111
230,Life on Land,2010,3,236,90,0.012712,0.033333
245,No Hunger,2010,2,236,90,0.008475,0.022222
291,Partnership for the Goals,2010,14,236,90,0.059322,0.155556
326,"Peace, Justice and Strong Institutions",2010,3,236,90,0.012712,0.033333


In [54]:
df_agg_final.Goal.value_counts()

Quality Education                          52
Industry, Innovation and Infrastructure    45
Decent Work and Economic Growth            44
Good Health and Well-Being                 44
Peace, Justice and Strong Institutions     40
Sustainable Cities and Communities         37
Reduced Inequalities                       31
Gender Equality                            20
Climate Action                             14
Partnership for the Goals                  13
Life on Land                                9
Zero Hunger                                 9
Responsible Consumption and Production      8
Life Below Water                            8
Affordable and Clean Energy                 4
No Poverty                                  3
Clean Water and Sanitation                  3
Name: Goal, dtype: int64

In [55]:
df_agg_final.Goal=df_agg_final.Goal.replace({'Zero Hunger':'No Hunger'})

In [57]:
df_agg_final.to_json(outputs_path+"/sg_visualization/proportions.json",orient="records")